In [1]:
# magic/dataset.py

import json
import logging
import os

from PIL import Image
from torch.utils.data import Dataset


class MAGICDataset(Dataset):
    """_summary_

    :param _type_ Dataset: MAGICDataset for ImageCLEF 2024 challenge
    """
    def __init__(self, file_path:str="data/", split:str="train"):
        """
        :param split: which dataset should be chosen
        :param file_path: main path with data
        """
        self.json_file = file_path + split + "_downloaded.json"
        self.folder_path = file_path + "images/" + split
        self.data = self._get_preprocessed_data()

    def _get_preprocessed_data(self):
        with open(self.json_file, encoding="utf8") as f :
            json_data = json.load(f)
        temp_data = []
        for sample in json_data:
            if len(sample["image_ids"]) != 1 :
                logging.warning(f'Different number of images ({len(sample["image_ids"])}) for question than 1')
            image_path = self.folder_path + '/' + sample["image_ids"][0] + '.jpg'
            if not os.path.exists(image_path):
                image_path = self.folder_path + '/' + sample["image_ids"][0] + '.png'
                if not os.path.exists(image_path):
                    logging.warning(f"Couldn't find path {image_path}")
                    continue
            temp_data.append({
                "image" : image_path,
                "description" : sample["query_title_en"],
                "answer" : sample["responses"][0]["content_en"],
                "encounter_id" : sample["encounter_id"]
            })
        return temp_data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample = self.data[idx]
        prompt = (
            "This is additional information about the dermatology issue on the image: "
            + sample["description"]
            + " What dermatological disease is on the image and how can it be treated?"
        )
        return {
            "image": Image.open(sample["image"]),  # Should be a PIL image
            "qa": [
                {
                    "question": prompt,
                    "answer": sample["answer"],
                }
            ], ## Why array?
            "encounter_id": sample['encounter_id']
        }

In [2]:
!pip install einops

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 548.6 kB/s eta 0:00:00


In [ ]:
# moondream2/inference.py

In [3]:
def print_sample_info(encounter_id, question, answer):
    print("-------------")
    print(f"{encounter_id=}")
    print(f"{question=}")
    print(f"{answer=}")
    print("-------------")

In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_id = "vikhyatk/moondream2"
revision = "2024-04-02"
# DEVICE = "cuda"
model = AutoModelForCausalLM.from_pretrained(
    "/content/drive/MyDrive/CLEF/models/model_1-10_epochs/",
    trust_remote_code=True,
    revision=revision,
    # device_map={"": DEVICE}
)
tokenizer = AutoTokenizer.from_pretrained(model_id, revision=revision)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


configuration_moondream.py:   0%|          | 0.00/3.41k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/vikhyatk/moondream2:
- configuration_moondream.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


moondream.py:   0%|          | 0.00/5.26k [00:00<?, ?B/s]

modeling_phi.py:   0%|          | 0.00/48.0k [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
model.device

device(type='cpu')

In [6]:
print("START")
dataset = MAGICDataset("/content/drive/MyDrive/reddit/", "valid")

response = []
for sample in dataset:
    encounter_id = sample["encounter_id"]
    question = sample['qa'][0]['question']
    encoded_image = model.encode_image(sample['image'])
    answer = model.answer_question(encoded_image, question, tokenizer)
    result = {
        "encounter_id": encounter_id,
        "responses": [{
            "content_en": answer
        }]
    }
    response.append(result)
    with open('/content/drive/MyDrive/CLEF/predictions/moondream2/2025-05-02-valid-prediction.json', 'w') as f:
        json.dump(response, f)
    print_sample_info(encounter_id, question, answer)

START


-------------
encounter_id='11r7ku3'
question='This is additional information about the dermatology issue on the image: Why do I have this raised horizontal skin/bump? on my forearm What dermatological disease is on the image and how can it be treated?'
answer='Most probably it is a case of friction blisters. Friction blisters are caused due to excessive friction between the skin and a surface. Examples of surfaces that could cause friction blisters include rough clothing, bed linens, or towels. They can also be caused by repetitive rubbing of the same area, such as when a person with hairy arms brushes his or her arms against a rough surface. Blisters usually heal on their own in a few weeks, but there are ways you can speed up the healing process. Apply a moisturizing cream containing aloe vera, petroleum jelly or shea butter. Wear loose cotton clothing. Avoid friction by wearing gloves when doing household chores.'
-------------
-------------
encounter_id='11b60q2'
question='This is